In [ ]:
import requests
import json
from flask import Flask, jsonify

#config api
API_CONFIG_FILE = "api.txt" #for full url
API_KEY_FILE = "apikey.txt" #for the api key only
app = Flask(__name__)

#reader for files
def read_file(filename):
    """reads a file, pretty simple"""
    try:
        with open(filename, 'r') as f:
            content = f.read().strip()
        if not content:
            return None, f"err: {filename} is empty."
        return content, None
    except FileNotFoundError:
        return None, f"err: {filename} not found."
    except Exception as e:
        return None, f"err: reading {filename}: {e}"

#get scores from api
def get_scores():
    """
    fetches scores or an error msg
    returns -> (info, matches, error)
    """
    #get url
    api_url, error = read_file(API_CONFIG_FILE)
    if error:
        return None, None, error
        
    #get  key
    api_key, error = read_file(API_KEY_FILE)
    if error:
        return None, None, error

    #prep api request
    params = {
        "apikey": api_key,
        "offset": 0 
    }

    try:
        #make request
        response = requests.get(api_url, timeout=10, params=params)

        #check for http errors
        if response.status_code != 200:
            return None, None, f"http error: {response.status_code} - {response.text}"

        #response to json
        data = response.json()

        #check api status
        if not (data.get('status') == 'success' and 'data' in data):
            reason = data.get('reason', 'unknown api error')
            return None, None, f"api error: {reason}"
        
        #get api status
        status_info = {}
        for key, value in data.items():
            if key not in ('data', 'apikey'):
                status_info[key] = value

        matches = data.get('data', [])
        return status_info, matches, None

    #catch network exception
    except requests.exceptions.RequestException as e:
        return None, None, f"network error: couldn't connect. {e}"
    except requests.exceptions.JSONDecodeError:
        return None, None, "err: bad json from api."
    except Exception as e:
        return None, None, f"an unexpected error: {e}"

#flask web
@app.route('/')
def show_scores():
    """main page, shows the json"""
    status_info, matches, error = get_scores()
    
    #josn error
    if error:
        # send a json error
        response = jsonify({"error": error})
        response.status_code = 500
        return response
    
    #program works
    else:
        #get data in web using http://127.0.0.1:5000/
        return jsonify({
            "api_status": status_info,
            "matches": matches
        })

#run
if __name__ == '__main__':
    #crash check
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.128:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2025 13:40:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:40:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:40:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:41:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:41:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2025 13:41:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:41:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 13:41:44] "GET / HTTP/1.1" 200 -
